<a href="https://colab.research.google.com/github/CynYZhu/MIDS_266_FinalProject/blob/main/266_Project_Goding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# W266 Project

In [ ]:
#install tensorflow datasets library
!pip install datasets
!pip install huggingface-hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 14.5 MB/s 
     |████████████████████████████████| 140 kB 88.5 MB/s 
     |████████████████████████████████| 1.1 MB 85.5 MB/s 
     |████████████████████████████████| 101 kB 11.3 MB/s 
     |████████████████████████████████| 212 kB 84.2 MB/s 
     |████████████████████████████████| 596 kB 83.3 MB/s 
     |████████████████████████████████| 127 kB 97.3 MB/s 
     |████████████████████████████████| 144 kB 90.6 MB/s 
     |████████████████████████████████| 94 kB 4.5 MB/s 
     |████████████████████████████████| 271 kB 71.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
ERROR: pip's de

In [ ]:
from datasets import load_dataset
from datasets import Dataset
import tensorflow as tf
import tensorflow_datasets as tfds
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from google.colab import files
import re

In [ ]:
dataset_train = load_dataset('trivia_qa', 'rc', split = 'train[:20%]')
#dataset_train, dataset_validation, dataset_test = load_dataset('trivia_qa', 'rc',split = ['train', 'validation', 'test'])

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/138384 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/17944 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/17210 [00:00<?, ? examples/s]

Dataset trivia_qa downloaded and prepared to /root/.cache/huggingface/datasets/trivia_qa/rc/1.2.0/e73c5e47a8704744fa9ded33504b35a6c098661813d1c2a09892eb9b9e9d59ae. Subsequent calls will reuse this data.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!ls drive/MyDrive/w266/data

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
cleaned_test.json  cleaned_traindata1.json  cleaned_validation.json


In [ ]:
dataset_test[0].keys()

In [ ]:
dataset_validation[0].keys()

dict_keys(['question', 'question_id', 'question_source', 'entity_pages', 'search_results', 'answer'])

In [ ]:
dataset_train[0].keys()

dict_keys(['question', 'question_id', 'question_source', 'entity_pages', 'search_results', 'answer'])

In [ ]:
#check how many empty search_context values
counter = 0
for item in dataset_train:
  if item['search_results']['search_context'] != []:
    continue   
  else:
    counter +=1 

In [ ]:
print('There are ' + str(counter) + ' empty search_context in the dataset_train')

There are 65038 empty search_context in the dataset_train


In [ ]:
#total length of dataset_train
len(dataset_train)

138384

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!ls drive/MyDrive/w266/data

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
cleaned_test.json  cleaned_traindata1.json  cleaned_validation.json


In [ ]:
#function to clean text
def clean_text(text):
  text = text.lower()
  text = text.replace('\\n', ' ').replace('\n', ' ').replace('\t',' ').replace('\\', ' ')
  text = re.sub(r'[^a-zA-Z0-9]',' ',text)
  return text

In [ ]:
#function to format train, validation, and test data
def format_dataset(dataset):

  #extract needed information
  question_id = dataset['question_id']
  question = dataset['question']
  #clean up question text, context by lower case, removing unicode chars, removing stop words
  question = clean_text(question)
  #wiki_context = dataset['entity_pages']['wiki_context']
  search_context = dataset['search_results']['search_context']
  #evaluate each item in search_context list and make one large context
  total_context = []
  for item in search_context:
    item = clean_text(item)
    total_context.append(item) 

  #normalized aliases
  answer = dataset['answer']['normalized_aliases']
 
  #join search context to one large string
  total_context_joined = ','.join(total_context)
  
  #list for answer text and answer start char in context
  answer_text_list = []
  answer_start_list = []
  for item2 in answer:
    answer_string = str(item2)
    start_char = answer_string[0]
    start_index = total_context_joined.find(start_char)
    answer_text_list.append(answer_string)
    answer_start_list.append(start_index)


  #answer = answer_start(dataset, total_context_joined)

  return {'id': question_id,  'question': question, 'context': total_context_joined, 'answers': {'text': answer_text_list,'answer_start': answer_start_list}}


In [ ]:
! cd drive/MyDrive/w266/data

In [ ]:
len(dataset_train)

27677

In [ ]:
#clean first set of train data
cleaned_traindata =[]
for x in range(0,27677):
    if dataset_train[x]['search_results']['search_context'] != []:
      cleaned_traindata.append(format_dataset(dataset_train[x]))
    else:
      continue



In [ ]:
len(cleaned_traindata)

26498

In [ ]:
#convert to hugging face dataset and save as json
cleaned_traindata_hf = Dataset.from_pandas(pd.DataFrame(data = cleaned_traindata))
cleaned_traindata_hf.to_json('drive/MyDrive/w266/data/cleaned_traindata_compact.json')
#save RAM
# cleaned_traindata =[]
# cleaned_traindata_hf =[]

Creating json from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

2104818688

In [ ]:
# #clean second set of train data
# cleaned_traindata =[]
# for x in range(30000,60000):
#     if dataset_train[x]['search_results']['search_context'] != []:
#       cleaned_traindata.append(format_dataset(dataset_train[x]))
#     else:
#       continue

# #convert to hugging face dataset and save as json
# cleaned_traindata_hf = Dataset.from_pandas(pd.DataFrame(data = cleaned_traindata))
# cleaned_traindata_hf.to_json('drive/MyDrive/w266/data/cleaned_traindata2.json')
# cleaned_traindata =[]
# cleaned_traindata_hf =[]

In [ ]:
# #clean third set of train data
# cleaned_traindata =[]
# for x in range(60000,138384):
#     if dataset_train[x]['search_results']['search_context'] != []:
#       cleaned_traindata.append(format_dataset(dataset_train[x]))
#     else:
#       continue

# #convert to hugging face dataset and save as json
# cleaned_traindata_hf = Dataset.from_pandas(pd.DataFrame(data = cleaned_traindata))
# cleaned_traindata_hf.to_json('drive/MyDrive/w266/data/cleaned_traindata3.json')
# cleaned_traindata =[]
# cleaned_traindata_hf

In [ ]:
len(dataset_validation)

17944

In [ ]:
#repeat process for validation data

cleaned_validation =[]
for x in range(0,17944):
    if dataset_validation[x]['search_results']['search_context'] != []:
      cleaned_validation.append(format_dataset(dataset_validation[x]))
    else:
      continue



In [ ]:
cleaned_validation_hf = Dataset.from_pandas(pd.DataFrame(data = cleaned_validation))
cleaned_validation_hf.to_json('drive/MyDrive/w266/data/cleaned_validation.json')

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

758618054

In [ ]:
#repeat for dataset test
len(dataset_test)

17210

In [ ]:
cleaned_test =[]
for x in range(0,17210):
    if dataset_test[x]['search_results']['search_context'] != []:
      cleaned_test.append(format_dataset(dataset_test[x]))
    else:
      continue


In [ ]:
cleaned_test_hf = Dataset.from_pandas(pd.DataFrame(data = cleaned_traindata))
cleaned_test_hf.to_json('drive/MyDrive/w266/data/cleaned_test.json')

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

55352804